In [21]:
import numpy as np
import scipy as sp
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'

In [22]:
def plot_3D_points(X):
    
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], 'o', ms=1, color='magenta', label='3D points')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')

    plt.legend(loc="lower right")
    # fig.savefig(path, dpi=300)

    plt.show()
    

In [23]:
def plot_image_points_and_image(x, img, path, plt_img=False):

    fig = plt.figure(figsize=(10,8))

    plt.plot(x[0], x[1], 'D', color='blue', label='Image points')

    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('equal')
    plt.gca().invert_yaxis()
    plt.legend(loc="lower right")

    if plt_img:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    fig.savefig(path, dpi=300)
    plt.show()

In [24]:
def plot_image_points_projected_points_and_image(x_proj, x_img, img, path):

    fig = plt.figure(figsize=(10,8))

    plt.plot(x_img[0], x_img[1], 'D', color='cyan', label='Image points')
    plt.plot(x_proj[0], x_proj[1], 'o', color='magenta', label='Projected points')

    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.axis('equal')
    plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    fig.savefig(path, dpi=300)

    plt.show()

In [25]:
def find_mean_and_std(x, axis):
    mean = np.sum(x[axis,:]) / np.size(x,1)
    std =  (np.sum((x[axis,:] - mean)**2) / np.size(x,1))**0.5
    return mean, std

In [26]:
def normalise_x_and_y(img_pts):

    axis = 0
    x_mean, x_std = find_mean_and_std(img_pts, axis)

    axis = 1
    y_mean, y_std = find_mean_and_std(img_pts, axis)

    print('x_mean:', x_mean, '\nx_std:', x_std, '\ny_mean:', y_mean, '\ny_std:', y_std)

    N = np.array([[1/x_std, 0, -x_mean/x_std],
                    [0, 1/y_std, -y_mean/y_std],
                    [0, 0, 1]])
    
    img_pts_norm = N @ img_pts

    return img_pts_norm, N

In [27]:
def estimate_camera_DLT(Xmodel, img_pts):

    n = np.size(img_pts,1)
    M = []

    for i in range(n):

        X = Xmodel[0,i]
        Y = Xmodel[1,i]
        Z = Xmodel[2,i]

        x = img_pts[0,i]
        y = img_pts[1,i]

        m = np.array([[X, Y, Z, 1, 0, 0, 0, 0, -x*X, -x*Y, -x*Z, -x],
                      [0, 0, 0, 0, X, Y, Z, 1, -y*X, -y*Y, -y*Z, -y]])

        M.append(m)

    M = np.concatenate(M, 0)
    U, S, VT = np.linalg.svd(M, full_matrices=False)
    P = np.stack([VT[-1, i:i+4] for i in range(0, 12, 4)], 0)
    # print(np.shape(U), '\n\n',np.shape(S), '\n\n', np.shape(VT))
    M_approx = U @ np.diag(S) @ VT

    v = VT[-1,:] # last row of VT because optimal v should be last column of V
    Mv = M @ v
    print('||Mv||:', (Mv @ Mv)**0.5)
    print('||v||^2:', v @ v)
    print('max{||M - M_approx||}:', np.max(np.abs(M - M_approx)))
    print('S:', S)

    return P

In [28]:
def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, path):
    
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], 'o', color='magenta', label='3D points')
    cv.plot_cameras_and_axes(ax, C_arr, axis_arr, s)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.axis('equal')

    plt.legend(loc="lower right")
    fig.savefig(path, dpi=300)

    plt.show()

Goal: estimate the 2 cameras using DLT
1. Normalize 
2. DLT
3. Project Xmodel on the images using P
4. Compute inner parameters of P1 using RQ-decomposition

In [29]:
def rq(a):

    m, n = a.shape
    e = np.eye(m)
    p = e[:, ::-1]
    q0, r0 = np.linalg.qr(p @ a[:, :m].T @ p)

    r = p @ r0.T @ p
    q = p @ q0.T @ p

    fix = np.diag(np.sign(np.diag(r)))
    r = r @ fix
    q = fix @ q

    if n > m:
        q = np.concatenate((q, np.linalg.inv(r) @ a[:, m:n]), axis=1)

    return r, q

In [30]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-2/'
data = r'A2data\data\compEx3data.mat'

Xmodel = cv.convert_mat_to_np(path+data, 'Xmodel')
endind = cv.convert_mat_to_np(path+data, 'endind')
startind = cv.convert_mat_to_np(path+data, 'startind')
x = cv.convert_mat_to_np(path+data, 'x')
img_pts_1 = x[0,0]
img_pts_2 = x[0,1]

img1 = r'A2data\data/cube1.jpg'
cube1 = cv.load_image(path+img1)

img2 = r'A2data\data/cube2.jpg'
cube2 = cv.load_image(path+img2)

In [34]:
# plot_3D_points(Xmodel)
# name = r'report-images/E2_test'
# plot_image_points_and_image(img_pts_1, cube1, path+name, plt_img=True)
# plot_image_points_and_image(img_pts_2, cube2, path+name, plt_img=True)


img_pts_norm, N = normalise_x_and_y(img_pts_1)
P_norm = estimate_camera_DLT(Xmodel, img_pts_norm)
P = np.linalg.inv(N) @ P_norm
np.save(r'A2data\data/P1.npy', P)

K, R = rq(P)
K_norm = K/K[-1,-1]
print('K:', K_norm)

x_proj = cv.dehomogenize(P @ cv.homogenize(Xmodel, multi=True))

name = r'report-images/E2_norm_img_pts_1'
# plot_image_points_and_image(img_pts_norm, cube1, path+name)

name = r'report-images/E2_proj_img_1'
# plot_image_points_projected_points_and_image(x_proj, img_pts_1, cube1, path+name)

C1_arr, axis1_arr = cv.compute_camera_and_normalized_principal_axis(P, multi=False)
s = 30
name = r'report-images/E2_3D_1'
# plot_cameras_and_3D_points(Xmodel, C_arr.T, axis_arr.T, s, path+name)




img_pts_norm, N = normalise_x_and_y(img_pts_2)
P_norm = estimate_camera_DLT(Xmodel, img_pts_norm)
P = np.linalg.inv(N) @ P_norm
np.save(r'A2data\data/P2.npy', P)

# K, R = rq(P)
# print('K:', K/K[-1,-1])

x_proj = cv.dehomogenize(P @ cv.homogenize(Xmodel, multi=True))

name = r'report-images/E2_norm_img_pts_2'
# plot_image_points_and_image(img_pts_norm, cube2, path+name)

name = r'report-images/E2_proj_img_2'
# plot_image_points_projected_points_and_image(x_proj, img_pts_2, cube2, path+name)

C2_arr, axis2_arr = cv.compute_camera_and_normalized_principal_axis(P, multi=False)
s = 30
name = r'report-images/E2_3D_2'




C_arr = np.concatenate((C1_arr, C2_arr),1)
axis_arr = np.concatenate((axis1_arr, axis2_arr),1)

name = r'report-images/E2_3D'
# plot_cameras_and_3D_points(Xmodel, C_arr, axis_arr, s, path+name)

print(np.size(cube1,0)/2, np.size(cube1,1)/2, K_norm[0,-1], K_norm[1,-1],)

x_mean: 1014.8534437663469 
x_std: 193.9348494348486 
y_mean: 839.0382565664256 
y_std: 195.8207116811975
||Mv||: 0.051034141134844696
||v||^2: 1.000000000000001
max{||M - M_approx||}: 1.2434497875801753e-14
S: [5.16418777e+01 3.57149336e+01 2.60836504e+01 1.78960893e+01
 1.46977672e+01 1.23441038e+01 9.73850317e+00 9.59219784e+00
 8.47436814e+00 3.81267118e+00 2.63790186e+00 5.10341411e-02]
K: [[ 2.42183633e+03 -6.77588216e+00  9.80194001e+02]
 [ 0.00000000e+00  2.42003888e+03  6.93975417e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
1
x_mean: 930.9657367044463 
x_std: 195.71450086643424 
y_mean: 795.1859535591931 
y_std: 196.72253529302543
||Mv||: 0.043782183046001934
||v||^2: 1.0
max{||M - M_approx||}: 1.5987211554602254e-14
S: [5.16002787e+01 3.57835001e+01 2.61367797e+01 1.79087519e+01
 1.46981725e+01 1.22099947e+01 9.71292239e+00 9.14699484e+00
 8.97383581e+00 3.79603268e+00 2.64283307e+00 4.37821830e-02]
1
648.0 968.0
